In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from lib.Exercise1_1 import LQRSolver
from torch.utils.data import TensorDataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import os
from datetime import datetime

Proj_dtype = torch.double
Proj_device = 'cpu' 


In [12]:
class DGMhiddenlayerYYBver(nn.Module):

    # From the original paper of Justin's, presented by Yuebo Yang Apr.9th 2024
    
    def __init__(self, input_f, output_f, activation = 'tanh'):
        
        super(DGMhiddenlayerYYBver, self).__init__()

        self.input_f = input_f
        self.output_f = output_f

        # Params

        # Zl's

        self.Uzl = nn.Parameter(torch.Tensor(output_f, input_f))
        self.Wzl = nn.Parameter(torch.Tensor(output_f, output_f))
        self.bzl = nn.Parameter(torch.Tensor(output_f))

        # Gl's

        self.Ugl = nn.Parameter(torch.Tensor(output_f, input_f))
        self.Wgl = nn.Parameter(torch.Tensor(output_f, output_f))
        self.bgl = nn.Parameter(torch.Tensor(output_f))

        # Rl's

        self.Url = nn.Parameter(torch.Tensor(output_f, input_f))
        self.Wrl = nn.Parameter(torch.Tensor(output_f, output_f))
        self.brl = nn.Parameter(torch.Tensor(output_f))

        # Hl's

        self.Uhl = nn.Parameter(torch.Tensor(output_f, input_f))
        self.Whl = nn.Parameter(torch.Tensor(output_f, output_f))
        self.bhl = nn.Parameter(torch.Tensor(output_f))


        if activation == 'tanh':
            self.activation = torch.tanh
        else:
            self.activation = None 

        self.init_method = 'normal' # or 'uniform'

        self._initialize_params()

    def _initialize_params(self):

        if self.init_method == 'uniform':
            for param in self.parameters():
                if param.dim() > 1:
                    init.xavier_uniform_(param)  
                else:
                    init.constant_(param, 0) 
                    
        if self.init_method == 'normal':
            for param in self.parameters():
                if param.dim() > 1:
                    init.xavier_normal_(param)  
                else:
                    init.constant_(param, 0) 

    def forward(self, x, S1, Sl):

        Zl = self.activation(torch.mm(x, self.Uzl.t())+ torch.mm(Sl, self.Wzl.t()) + self.bzl)

        Gl = self.activation(torch.mm(x, self.Ugl.t())+ torch.mm(S1, self.Wgl.t()) + self.bgl)

        Rl = self.activation(torch.mm(x, self.Url.t())+ torch.mm(Sl, self.Wrl.t()) + self.brl)

        Hl = self.activation(torch.mm(x, self.Uhl.t())+ torch.mm(torch.mul(Sl,Rl), self.Whl.t()) + self.bhl)

        Sl_1 = torch.mul((1-Gl),Hl) + torch.mul(Zl,Sl)

        return Sl_1



In [13]:

class DGMNN_YYBver(nn.Module):

    # From the original paper of Justin's, presented by Yuebo Yang Apr.9th 2024

    def __init__(self, init_method = 'uniform'):
        super(DGMNN_YYBver, self).__init__()

        self.nodenum = 50

        self.layer1 = DGMhiddenlayerYYBver(3, self.nodenum)
        self.layer2 = DGMhiddenlayerYYBver(3, self.nodenum)
        self.layer3 = DGMhiddenlayerYYBver(3, self.nodenum)

        self.tanh = nn.Tanh()

        # Params

        # S1's

        self.W1 = nn.Parameter(torch.Tensor(self.nodenum, 3))
        self.b1 = nn.Parameter(torch.Tensor(self.nodenum))

        # Output's

        self.W = nn.Parameter(torch.Tensor(1, self.nodenum))
        self.b = nn.Parameter(torch.Tensor(1))

        self.activation = torch.tanh

        self.init_method = 'normal' # or 'uniform'

        self._initialize_params()

    def _initialize_params(self):

        if self.init_method == 'uniform':
            for param in self.parameters():
                if param.dim() > 1:
                    init.xavier_uniform_(param)  
                else:
                    init.constant_(param, 0) 
                    
        if self.init_method == 'normal':
            for param in self.parameters():
                if param.dim() > 1:
                    init.xavier_normal_(param)  
                else:
                    init.constant_(param, 0) 

        
    def forward(self, x):

        S_1 = self.activation(torch.mm(x, self.W1.t()) + self.b1)
        # l=1
        S_2 = self.layer1(x,S_1,S_1)
        # l=2
        S_3 = self.layer2(x,S_1,S_2)
        # l=3
        S_4 = self.layer3(x,S_1,S_3)

        output = torch.mm(S_4, self.W.t()) + self.b

        return output


In [14]:

class ControlNN_YYBver(nn.Module):

    # From the original paper of Justin's, presented by Yuebo Yang Apr.9th 2024

    def __init__(self, init_method = 'uniform'):
        super(ControlNN_YYBver, self).__init__()

        self.nodenum = 50

        self.layer1 = DGMhiddenlayerYYBver(3, self.nodenum)
        self.layer2 = DGMhiddenlayerYYBver(3, self.nodenum)
        self.layer3 = DGMhiddenlayerYYBver(3, self.nodenum)

        self.tanh = nn.Tanh()

        # Params

        # S1's

        self.W1 = nn.Parameter(torch.Tensor(self.nodenum, 3))
        self.b1 = nn.Parameter(torch.Tensor(self.nodenum))

        # Output's

        self.W = nn.Parameter(torch.Tensor(2, self.nodenum))
        self.b = nn.Parameter(torch.Tensor(2))

        self.activation = torch.tanh

        self.init_method = 'normal' # or 'uniform'

        self._initialize_params()

    def _initialize_params(self):

        if self.init_method == 'uniform':
            for param in self.parameters():
                if param.dim() > 1:
                    init.xavier_uniform_(param)  
                else:
                    init.constant_(param, 0) 
                    
        if self.init_method == 'normal':
            for param in self.parameters():
                if param.dim() > 1:
                    init.xavier_normal_(param)  
                else:
                    init.constant_(param, 0) 

        
    def forward(self, x):

        S_1 = self.activation(torch.mm(x, self.W1.t()) + self.b1)
        # l=1
        S_2 = self.layer1(x,S_1,S_1)
        # l=2
        S_3 = self.layer2(x,S_1,S_2)
        # l=3
        S_4 = self.layer3(x,S_1,S_3)

        output = torch.mm(S_4, self.W.t()) + self.b

        return output


In [15]:
def get_hessian(grad,x):
    Hessian = torch.tensor([], device = Proj_device)
    
    for i in range(len(x)):
        hessian = torch.tensor([], device = Proj_device)
        for j in range(len(grad[i])):
            u_xxi = torch.autograd.grad(grad[i][j], x, grad_outputs=torch.ones_like(grad[i][j]), retain_graph=True,create_graph=True, allow_unused=True)[0]           
            hessian = torch.cat((hessian, u_xxi[i].unsqueeze(0)))
        Hessian = torch.cat((Hessian, hessian.unsqueeze(0)),dim = 0)
        # print(Hessian)
    return Hessian

def get_hessian_(model,t,x):
    Hessian = torch.tensor([], device = Proj_device)
    for i in range(len(t)):
        x_i = V(x[i],requires_grad=True)
        input = torch.cat(((t[i]).unsqueeze(0), x_i),dim=0)
        u_in = model(input)
        grad = torch.autograd.grad(u_in, x_i, grad_outputs=torch.ones_like(u_in), create_graph=True, retain_graph=True)[0]
        hessian = torch.tensor([], device = Proj_device)
        for j in range(len(grad)):
            u_xxi = torch.autograd.grad(grad[j], x_i, grad_outputs=torch.ones_like(grad[j]), retain_graph=True,create_graph=True, allow_unused=True)[0]           
            hessian = torch.cat((hessian, u_xxi.unsqueeze(0)))
        Hessian = torch.cat((Hessian, hessian.unsqueeze(0)),dim = 0)
    return Hessian

def pde_residual(model, t, x, H, M, sigma, alpha, C, D):
    
    input = torch.cat((t.unsqueeze(1), x),dim=1)
    
    u = model(input)

    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]

    u_xx = get_hessian(u_x,x)

#    u_xx = get_hessian_(model,t,x)
    
    residual = u_t + 0.5 * torch.einsum('bii->b', sigma @ sigma.transpose(1,2) @ u_xx) + (u_x.unsqueeze(1) @ (H @ x.unsqueeze(1).transpose(1,2)) + u_x.unsqueeze(1) @ M @ alpha.unsqueeze(1).transpose(1,2) + x.unsqueeze(1) @ C @ x.unsqueeze(1).transpose(1,2) + alpha.unsqueeze(1) @ D @ alpha.unsqueeze(1).transpose(1,2)).squeeze()
    
    return residual

def boundary_condition(model, t, x, R, T):

    
    T_input = T * torch.ones_like(t)

    input = torch.cat((T_input.unsqueeze(1), x),dim=1)
    u = model(input)

    return u - (x.unsqueeze(1) @ R @ x.unsqueeze(1).transpose(1,2)).squeeze()

def total_residual(model, t, x, H, M, sigma, alpha, C, D, R, T):
    
    residual_loss = pde_residual(model, t, x, H, M, sigma, alpha, C, D).pow(2).mean()
    boundary_loss = boundary_condition(model, t, x, R, T).pow(2).mean()
    
    return residual_loss + boundary_loss


In [16]:


def new_data(T, num_samples):
    # num_samples = 1000
    t_samples = T * torch.rand(num_samples, dtype=Proj_dtype, device = Proj_device, requires_grad=False)
    x_ends = torch.tensor([-3,3], dtype = Proj_dtype)
    x_samples = x_ends[0] + (x_ends[1]- x_ends[0]) * torch.rand(num_samples , 2, dtype=Proj_dtype, device = Proj_device, requires_grad=False)
    return t_samples,x_samples

In [17]:
# Define matrices for LQR problem

H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]], dtype=Proj_dtype, device = Proj_device)
M = torch.tensor([[0.5,0.7], [0.3,1.0]], dtype=Proj_dtype, device = Proj_device)
sigma = torch.tensor([[[0.08],[0.11]]], dtype=Proj_dtype, device = Proj_device)

C = torch.tensor([[1.6, 0.0], [0.0, 1.1]], dtype=Proj_dtype, device = Proj_device)
D = torch.tensor([[0.5, 0.0], [0.0, 0.7]], dtype=Proj_dtype, device = Proj_device)
R = torch.tensor([[0.9, 0.0], [0.0, 1.0]], dtype=Proj_dtype, device = Proj_device)
T = torch.tensor(1.0, dtype=Proj_dtype, device = Proj_device)

solver = LQRSolver(H, M, sigma, C, D, R, T=T, method="euler")


In [18]:

t_samples, x_samples = new_data(T, 1000)


def get_data_by_index(iter_p, t_samples, x_samples):
    """
    根据索引o获取对应的t_data和x_data。
    
    参数:
    - o: 索引，从1开始计数。
    - t_samples: 包含t数据的张量。
    - x_samples: 包含x数据的张量。
    
    返回:
    - t_data: 对应索引的t数据。
    - x_data: 对应索引的x数据。
    """
    # 确保o从1开始，因此需要减1来匹配Python的索引从0开始
    index = iter_p - 1
    # 获取对应索引的数据
    t_data = t_samples[index]
    x_data = x_samples[index]
    return t_data, x_data

# 使用示例
o = 1  # 假设你想获取第一个样本的数据
t_data, x_data = get_data_by_index(o, t_samples, x_samples)
print(f"Sample {o}: t_data = {t_data}, x_data = {x_data}")


Sample 1: t_data = 0.7591819184958559, x_data = tensor([-2.0261,  2.9040], dtype=torch.float64)


In [11]:
t_t,x_t = new_data(T,5)

In [13]:
x_t

tensor([[-1.2856, -2.5480],
        [-1.5361, -2.1179],
        [ 2.9114, -2.4562],
        [-0.8773, -0.3769],
        [ 0.0379, -0.8511]], dtype=torch.float64)

In [14]:
control_DGM_test = ControlNN_YYBver().double()
input = torch.cat((t_t.unsqueeze(1), x_t),dim=1)

a_t = control_DGM_test(input)


In [15]:
a_t

tensor([[-0.0640,  0.7670],
        [ 0.3452,  0.8818],
        [ 0.2939, -0.9116],
        [-0.8331, -0.0436],
        [ 0.0493, -0.0433]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [25]:
alpha = alpha.transpose(1,2).squeeze(0)
alpha

tensor([[1., 1.]], dtype=torch.float64)

In [19]:
a_t.unsqueeze(1) @ D @ a_t.unsqueeze(1).transpose(1,2)

tensor([[[0.4138]],

        [[0.6039]],

        [[0.6250]],

        [[0.3484]],

        [[0.0025]]], dtype=torch.float64, grad_fn=<UnsafeViewBackward0>)

In [26]:
alpha.unsqueeze(1) @ D @ alpha.unsqueeze(1).transpose(1,2)

tensor([[[1.2000]]], dtype=torch.float64)

In [21]:
x_t.unsqueeze(1) @ C @ x_t.unsqueeze(1).transpose(1,2)

tensor([[[ 9.7858]],

        [[ 8.7090]],

        [[20.1982]],

        [[ 1.3879]],

        [[ 0.7991]]], dtype=torch.float64)

In [22]:
x_t.unsqueeze(1) @ C @ x_t.unsqueeze(1).transpose(1,2) + alpha.transpose(1,2) @ D @ alpha

tensor([[[10.9858]],

        [[ 9.9090]],

        [[21.3982]],

        [[ 2.5879]],

        [[ 1.9991]]], dtype=torch.float64)

In [18]:
alpha.transpose(1,2) @ D @ alpha

tensor([[[1.2000]]], dtype=torch.float64)

In [19]:
#Update of value function

def value_update(H, M, sigma, alpha, C, D, R, T): 

    model_DGM = DGMNN_YYBver().double()
    optimizer_DGM = torch.optim.Adam(model_DGM.parameters(), lr=0.0001)
    scheduler_DGM = lr_scheduler.ExponentialLR(optimizer_DGM, gamma=0.9)


    continue_training = input("Do you want to continue training or start a new one? (c/n): ").lower() == 'c'

    model_save_path = 'Exercise4/model_DGM_state_dict.pt'
    optimizer_save_path = 'Exercise4/optimizer_DGM_state.pt'

    if continue_training and os.path.exists(model_save_path) and os.path.exists(optimizer_save_path):
        
        model_DGM.load_state_dict(torch.load(model_save_path))
        optimizer_DGM.load_state_dict(torch.load(optimizer_save_path))
        print("Continuing training from saved state.")
    else:
        print("Starting training from scratch.")

    epoch_losses = []

    iterations = 50
    epochs = 100

    patience = 10

    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    filename = f'Exercise4/value_training_loss_{timestamp}.dat'

    with open(filename, 'w') as f:

        for iteration in range(iterations):
            print(f'Iteration {iteration+1}/{iterations}'+'\n')
            
            dataset = TensorDataset(t_data,x_data)
            dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

            best_loss = float('inf')
            patience_counter = 0  

            for epoch in range(epochs):

                model_DGM.train()
                total_loss = 0
                
                for batch_idx, (_t_data,_x_data) in enumerate(dataloader):
                    optimizer_DGM.zero_grad()
                    t_data = _t_data.clone().requires_grad_(True)
                    x_data = _x_data.clone().requires_grad_(True)
                    loss = total_residual(model_DGM, t_data, x_data, H, M, sigma, alpha, C, D, R, T) 
                    loss.backward()
                    optimizer_DGM.step()
                    total_loss += loss.item()
                
                avg_loss = total_loss / len(dataloader)
                epoch_losses.append(avg_loss)
                
                scheduler_DGM.step()

                f.write(f'Iteration {iteration+1}, Epoch {epoch+1}, Loss: {avg_loss}\n')

                if avg_loss < best_loss:
                    best_loss = avg_loss
                    patience_counter = 0  
                    torch.save(model_DGM.state_dict(), model_save_path)
                    torch.save(optimizer_DGM.state_dict(), optimizer_save_path)
                else:
                    patience_counter += 1

                if epoch == 0 or (epoch+1) % 5 == 0:
                    print(f'Epoch {epoch+1}/{epochs} \t Loss: {avg_loss}')
                
                
                if patience_counter >= patience:
                    print(f'Early stopping triggered at epoch {epoch+1}')
                    break  
            print('\n')
    return  model_DGM(torch.cat((t_data.unsqueeze(1), x_data),dim=1)).squeeze()     


In [20]:
#Hamiltonian

def Hamiltonian(alpha, current_value, x, H, M, C, D):


    u = current_value
    a = alpha


    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]

    H_i  = u_x.unsqueeze(1) @ H @ x.unsqueeze(1).transpose(1,2) + u_x.unsqueeze(1) @ M @ a.unsqueeze(1).transpose(1,2) + x.unsqueeze(1) @ C @ x.unsqueeze(1).transpose(1,2) + a.unsqueeze(1) @ D @ a.unsqueeze(1).transpose(1,2)
    
    #  The true hamiltonian
    H  = H_i.mean()

In [21]:
#Update of control

def control_update(alpha, current_value, x_data, H, M, C, D): 

    control_DGM = ControlNN_YYBver().double()
    optimizer_control = torch.optim.Adam(control_DGM.parameters(), lr=0.0001)
    scheduler_control = lr_scheduler.ExponentialLR(optimizer_control, gamma=0.9)


    continue_training = input("Do you want to continue training or start a new one? (c/n): ").lower() == 'c'

    model_save_path = 'Exercise4/control_DGM_state_dict.pt'
    optimizer_save_path = 'Exercise4/optimizer_control_state.pt'

    if continue_training and os.path.exists(model_save_path) and os.path.exists(optimizer_save_path):
        
        control_DGM.load_state_dict(torch.load(model_save_path))
        optimizer_control.load_state_dict(torch.load(optimizer_save_path))
        print("Continuing training from saved state.")
    else:
        print("Starting training from scratch.")

    epoch_losses = []

    iterations = 50
    epochs = 100

    patience = 10

    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    filename = f'Exercise4/control_training_loss_{timestamp}.dat'

    with open(filename, 'w') as f:

        for iteration in range(iterations):
            print(f'Iteration {iteration+1}/{iterations}'+'\n')
            

            dataset = TensorDataset(t_data,x_data)
            dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

            best_loss = float('inf')
            patience_counter = 0  

            for epoch in range(epochs):

                control_DGM.train()
                total_loss = 0
                
                for batch_idx, (_t_data,_x_data) in enumerate(dataloader):
                    optimizer_control.zero_grad()
                    t_data = _t_data.clone().requires_grad_(True)
                    x_data = _x_data.clone().requires_grad_(True)
                    loss = Hamiltonian(alpha, current_value, x_data, H, M, C, D) 
                    loss.backward()
                    optimizer_control.step()
                    total_loss += loss.item()
                
                avg_loss = total_loss / len(dataloader)
                epoch_losses.append(avg_loss)
                
                scheduler_control.step()

                f.write(f'Iteration {iteration+1}, Epoch {epoch+1}, Loss: {avg_loss}\n')

                if avg_loss < best_loss:
                    best_loss = avg_loss
                    patience_counter = 0  
                    torch.save(control_DGM.state_dict(), model_save_path)
                    torch.save(optimizer_control.state_dict(), optimizer_save_path)
                else:
                    patience_counter += 1

                if epoch == 0 or (epoch+1) % 5 == 0:
                    print(f'Epoch {epoch+1}/{epochs} \t Loss: {avg_loss}')
                
                
                if patience_counter >= patience:
                    print(f'Early stopping triggered at epoch {epoch+1}')
                    break  
            print('\n')
        
    return control_DGM(torch.cat((t_data.unsqueeze(1), x_data),dim=1)).squeeze()


In [23]:
# Policy Iteration

alpha = torch.tensor([[[1],[1]]], dtype=Proj_dtype, device = Proj_device)
for iter_p in range(999):
    current_value = value_update(H, M, sigma, alpha, C, D, R, T)
    alpha = control_update(alpha, current_value, x_data, H, M, C, D)
    t_data = t_samples[iter_p]
    x_data = x_samples[iter_p]



Starting training from scratch.


FileNotFoundError: [Errno 2] No such file or directory: 'Exercise4/value_training_loss_2024-04-10_18-22-47.dat'